In [1]:
import torch.nn as nn
import torch.nn.functional as F
import net_size_utils as nsu

class Net(nn.Module):
    def __init__(self, kernel_size=5, channel_sizes=None):
        super().__init__()
        if channel_sizes == None:
            channel_sizes = [6, 16]
        self.channel_sizes = channel_sizes
        self.kernel_size = kernel_size
        self.conv1 = nn.Conv2d(3, channel_sizes[0], kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(channel_sizes[0], channel_sizes[1], kernel_size)
        conv_height, conv_width = self.conv_out_size(224, 224)
        linear_input_size = channel_sizes[1] * conv_width * conv_height
        self.fc1 = nn.Linear(linear_input_size, 120)
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    def conv_out_size(self, height, width):
        dim = (height, width)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        dim = nsu.dim_conv2d(dim, self.kernel_size)
        dim = nsu.dim_maxpool2d(dim, 2, 2)
        return dim

In [2]:
import statistics
import torch.utils.data as td
import net_training
from birds_dataset import Birds270Dataset
import torch
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import pandas as pd

# Splits a dataset randomly into a train and test set. The size of test set is 80% of the whole dataset.
# Then it trains the network
def train_net_random_dataset_split(net, dataset, epochs, optimizer, batch_size):
    train_set_size = int(len(dataset)*0.8)
    test_set_size = len(dataset)-train_set_size
    train_dataset, test_dataset = td.random_split(dataset, [train_set_size, test_set_size])
    label_set = dataset.get_label_set()
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)
    results = net_training.train_and_evaluate(net, train_dataloader, test_dataloader, label_set,
                                 epochs=epochs, optimizer=optimizer, print_results=True)
    return results

# Creates a few networks and trains them using a random split of the dataset.
# The number of created networks is in the "repeat" argument
# It returns the final validation results for each network
def cross_validate_net(net_generator, dataset, repeat=5, epochs=20, batch_size=32):
    all_results = []
    for i in range(repeat):
        print(f"Training network {i+1} ...")
        net = net_generator()
#        optimizer = optim.SGD(net.parameters(), lr=0.001)
        results = train_net_random_dataset_split(net, dataset, epochs, optimizer=None, batch_size=batch_size)
        all_results.append(results)
        net_training.print_final_results(results)
    return all_results
    
    
def print_validation_results(cross_validation_results):
    losses = [r["loss"] for r in cross_validation_results]
    accuracies = [r["accuracy"] for r in cross_validation_results]
    print("Losses: ", losses)
    print("Loss:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(losses), statistics.stdev(losses)))
    print("Accuracies: ", accuracies)
    print("Accuracy:  mean: {:.4f}, std: {:.4f}".format(statistics.mean(accuracies), statistics.stdev(accuracies)))
    
def results_to_dataframe(cross_validation_results):
    normalized = pd.json_normalize(cross_validation_results)
    return normalized
    


In [3]:
dataset_dir = "../data/birds270"
selected_birds = ["ALBATROSS", "BALD EAGLE", "BARN OWL", "EURASIAN MAGPIE", "FLAMINGO",
                  "MALLARD DUCK", "OSTRICH", "PEACOCK", "PELICAN", "TRUMPTER SWAN"]

transform = transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5)) # normalizes colors to range [-1,1]
dataset = Birds270Dataset(dataset_dir,  selected_birds=selected_birds, transform=transform)

def net_generator():
    return Net(channel_sizes=[16,32])

results = cross_validate_net(net_generator, dataset, repeat=5, epochs=100, batch_size=64)
print_validation_results(results)

Training network 1 ...
Epoch 0:
	train loss: 2.023600456849584
	validation loss: 1.480298937183537, validation accuracy: 53.082191780821915%
	Elapsed time: 0:00:15.257022
Epoch 1:
	train loss: 1.0770292558146586
	validation loss: 1.0998152837361375, validation accuracy: 61.64383561643836%
	Elapsed time: 0:00:15.177414
Epoch 2:
	train loss: 0.6667725618666687
	validation loss: 0.9668851592769362, validation accuracy: 68.83561643835617%
	Elapsed time: 0:00:15.131430
Epoch 3:
	train loss: 0.39491381131968817
	validation loss: 1.0248783149131357, validation accuracy: 72.6027397260274%
	Elapsed time: 0:00:15.112058
Epoch 4:
	train loss: 0.24323910807050111
	validation loss: 0.9617468624898832, validation accuracy: 72.26027397260275%
	Elapsed time: 0:00:15.200569
Epoch 5:
	train loss: 0.07846553250348537
	validation loss: 0.9528589959013952, validation accuracy: 75.34246575342466%
	Elapsed time: 0:00:15.141162
Epoch 6:
	train loss: 0.022750074369753
	validation loss: 1.007152847231251, valid

Epoch 21:
	train loss: 0.0005766777799438404
	validation loss: 1.2673759754389933, validation accuracy: 75.34246575342466%
	Elapsed time: 0:00:15.119766
Epoch 22:
	train loss: 0.0005267275980351042
	validation loss: 1.2770135819095454, validation accuracy: 75.34246575342466%
	Elapsed time: 0:00:15.214666
Epoch 23:
	train loss: 0.0004796820959694505
	validation loss: 1.2908602969287193, validation accuracy: 75.0%
	Elapsed time: 0:00:15.070196
Epoch 24:
	train loss: 0.0004436160757705189
	validation loss: 1.301316450719964, validation accuracy: 75.34246575342466%
	Elapsed time: 0:00:15.133255
Epoch 25:
	train loss: 0.0004089692414128805
	validation loss: 1.3100830855434888, validation accuracy: 75.0%
	Elapsed time: 0:00:15.168721
Epoch 26:
	train loss: 0.00037910894359586563
	validation loss: 1.3161332615434307, validation accuracy: 75.0%
	Elapsed time: 0:00:15.201857
Epoch 27:
	train loss: 0.0003520009055420977
	validation loss: 1.3304269411792493, validation accuracy: 75.0%
	Elapsed ti

Epoch 1:
	train loss: 1.3986370135130646
	validation loss: 1.252012162992399, validation accuracy: 58.56164383561644%
	Elapsed time: 0:00:15.223650
Epoch 2:
	train loss: 0.8737828300093295
	validation loss: 0.9621897633761576, validation accuracy: 67.8082191780822%
	Elapsed time: 0:00:15.282429
Epoch 3:
	train loss: 0.5075309228529039
	validation loss: 0.963859009416136, validation accuracy: 65.41095890410958%
	Elapsed time: 0:00:15.177394
Epoch 4:
	train loss: 0.2617383903374905
	validation loss: 0.923355272371475, validation accuracy: 72.6027397260274%
	Elapsed time: 0:00:15.288957
Epoch 5:
	train loss: 0.1406730502415736
	validation loss: 1.0327518043452746, validation accuracy: 69.86301369863014%
	Elapsed time: 0:00:15.319197
Epoch 6:
	train loss: 0.061010875408481775
	validation loss: 1.1180271563464648, validation accuracy: 67.12328767123287%
	Elapsed time: 0:00:15.210281
Epoch 7:
	train loss: 0.025894684689902216
	validation loss: 1.030972764916616, validation accuracy: 71.57534

Epoch 27:
	train loss: 0.0005277627619785316
	validation loss: 2.314424374332167, validation accuracy: 66.0958904109589%
	Elapsed time: 0:00:15.564590
Epoch 28:
	train loss: 0.0004863024669887872
	validation loss: 2.3324785199883866, validation accuracy: 65.75342465753424%
	Elapsed time: 0:00:16.023238
Epoch 29:
	train loss: 0.0004554050251694578
	validation loss: 2.3441170600995624, validation accuracy: 66.0958904109589%
	Elapsed time: 0:00:16.339909
Epoch 30:
	train loss: 0.00042371751763937775
	validation loss: 2.362511984289509, validation accuracy: 65.75342465753424%
	Elapsed time: 0:00:16.520022
Epoch 31:
	train loss: 0.0003977686684541915
	validation loss: 2.3720707828051424, validation accuracy: 65.75342465753424%
	Elapsed time: 0:00:15.723131
Epoch 32:
	train loss: 0.0003715566871947358
	validation loss: 2.391214081685837, validation accuracy: 66.0958904109589%
	Elapsed time: 0:00:15.587368
Epoch 33:
	train loss: 0.0003519783045207574
	validation loss: 2.39720472570968, valida

In [4]:
dataframe = results_to_dataframe(results)
dataframe.to_csv("../results/net_2conv_2fc_channels_16_32.csv")